In [53]:
import pandas as pd
import os
from random import randint, sample
import re
import csv

In [9]:
print(os.getcwd())

/Users/maryjones/Elbrus/Study_files/ds-phase-2/11-nn-ext/find_my_book/notebooks


In [5]:
data = pd.read_csv('../data/books.csv')

In [22]:
data = data.iloc[8:]
data.head()

,title,author,url,image,description
8,Гимназисты. Истории о мальчиках XIX века (лими...,"Гарин-Михайловский Н.Г., Погорельский А., Авер...",https://www.biblio-globus.ru/product/11024372,https://static1.bgshop.ru/imagehandler.ashx?fi...,"Из чего же сделаны мальчишки XIX века? Может, ..."
9,Происхождение,Браун Д.,https://www.biblio-globus.ru/product/10447554,https://static1.bgshop.ru/imagehandler.ashx?fi...,Роберт Лэнгдон прибывает в музей Гуггенхайма в...
10,Такси до леса Берендея,Донцова Д.А.,https://www.biblio-globus.ru/product/11043229,https://static1.bgshop.ru/imagehandler.ashx?fi...,"Если женщина не хочет похудеть, значит, она ум..."
11,Пока воды Венеции тихие,Эрвас Ф.,https://www.biblio-globus.ru/product/11044003,https://static1.bgshop.ru/imagehandler.ashx?fi...,Атмосферный итальянский детектив. Дорога к сер...


In [33]:
# import requests
# from bs4 import BeautifulSoup

# BASE_URL = "https://www.biblio-globus.ru"
# START_URL = "https://www.biblio-globus.ru/catalog/categories"

# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
# }

# try:
#     response = requests.get(START_URL, headers=headers, timeout=30)
#     response.raise_for_status()
    
#     soup = BeautifulSoup(response.text, "html.parser")
    
#     # Находим все ссылки которые содержат /catalog/index/
#     catalog_links = soup.find_all('a', href=lambda x: x and x.startswith('/catalog/index/'))
    
#     subcategories = []
#     for link in catalog_links:
#         href = link.get('href')
#         text = link.get_text(strip=True)
#         if href and text:  # проверяем что есть и ссылка и текст
#             # Извлекаем номер индекса из конца ссылки
#             index_number = href.split('/')[-1]  # берем последнюю часть после /
#             full_url = BASE_URL + href
#             subcategories.append((text, full_url, index_number))
    
#     # Выводим результат
#     print(f"Найдено подкатегорий: {len(subcategories)}\n")
#     for name, url, index in subcategories:
#         print(f"{name} - {url} - Индекс: {index}")
        
# except Exception as e:
#     print(f"Ошибка: {e}")


In [ ]:
# links = pd.DataFrame(subcategories)
# links = links.iloc[:-66]
# links.to_csv('../data/subcategories.csv')

In [ ]:
# links = df = pd.read_csv("../data/subs.csv", header=None)
# links.columns = ["n", "0", "1", "2"]
# links = links.drop('n', axis=1)
# links = links.drop(index=0).reset_index(drop=True)
# links.head()
# links.to_csv("../data/subcategories.csv", index=False)


In [158]:
links = pd.read_csv("../data/subcategories.csv")
links.head()

,0,1,2
0,Классическая и современная проза,https://www.biblio-globus.ru/catalog/index/182,182
1,Детективы,https://www.biblio-globus.ru/catalog/index/185,185
2,Фантастика. Фэнтези,https://www.biblio-globus.ru/catalog/index/186,186
3,Любовные романы. Книги о любви,https://www.biblio-globus.ru/catalog/index/189,189
4,Поэзия. Драматургия,https://www.biblio-globus.ru/catalog/index/6172,6172


In [ ]:
# st_accept = "text/html" 
# st_useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"
# headers = {
#    "Accept": st_accept,
#    "User-Agent": st_useragent
# }
# req = requests.get("https://www.biblio-globus.ru/catalog/category?id=226", headers=headers)
# src = req.text

# with open("biblio_page_226.html", "w", encoding="utf-8") as f:
#     f.write(req.text)

In [25]:
def extract_product_ids(url):
    try:
        # Загружаем страницу
        response = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')   # Парсим HTML
        product_ids = []
        product_links = soup.find_all('a', href=lambda x: x and x.startswith('/product/'))
        
        for link in product_links:
            href = link.get('href')
            # Извлекаем номер продукта из ссылки
            product_id = href.replace('/product/', '').strip()
            if product_id and product_id not in product_ids:
                product_ids.append(product_id)
        
        return product_ids
        
    except Exception as e:
        print(f"Ошибка при загрузке страницы: {e}")
        return []

# # Использование
# product_ids = extract_product_ids('https://www.biblio-globus.ru/catalog/index/185')
# print("Найдены ID продуктов:", product_ids)

# # Полные ссылки
# full_urls = [f"https://www.biblio-globus.ru/product/{pid}" for pid in product_ids]
# for url in full_urls:
#     print(url)

In [183]:
# def pars_book_data(url, csv_filename="../data/books.csv"):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()
#     except Exception as e:
#         print(f"[ERROR] Не удалось загрузить страницу {url}: {e}")
#         return

#     soup = BeautifulSoup(response.text, "lxml")

#     title = (soup.find("meta", property="og:title") or {}).get("content", "")
#     author = (soup.find("meta", property="og:book:author") or {}).get("content", "")
#     url_value = (soup.find("meta", property="og:url") or {}).get("content", "")
#     image = (soup.find("meta", property="og:image") or {}).get("content", "")

#     description = ""
#     module_div = soup.find('div', id='module')
#     if module_div:
#         collapse_div = module_div.find('div', class_='collapse')
#         if collapse_div:
#             for table in collapse_div.find_all('table'):
#                 table.decompose()
#             for h3 in collapse_div.find_all('h3'):
#                 if 'Характеристики' in h3.get_text():
#                     h3.decompose()
#             description = collapse_div.get_text(strip=True, separator=' ')
#             description = ' '.join([line.strip() for line in description.split('\n') if line.strip()])

#     data = {
#         "title": title,
#         "author": author,
#         "url": url_value,
#         "image": image,
#         "description": description
#     }

#     fieldnames = ["title", "author", "url", "image", "description"]

#     current_dir = os.getcwd()
#     full_path = os.path.join(current_dir, csv_filename)
    
#     file_exists = os.path.isfile(full_path)

#     try:
#         with open(full_path, "a", newline="", encoding="utf-8") as csvfile:
#             writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#             if not file_exists:
#                 writer.writeheader()

#             writer.writerow(data)

#     except Exception as e:
#         print(f"[ERROR] Не удалось сохранить данные: {e}")

In [184]:
# pars_book_data('https://www.biblio-globus.ru/product/11021457')

In [182]:
# extract_product_ids('https://www.biblio-globus.ru/catalog/category?id=185&page=2')

In [ ]:
# import re, time, random, requests
# from bs4 import BeautifulSoup

# for i in range(1):      # количество заходов по категориям
#     n = random.randint(0, len(links) - 1)
#     subcategorie = links.iloc[n][2]

#     url_subcat = f'https://www.biblio-globus.ru/catalog/category?id={subcategorie}'
#     html = requests.get(url_subcat).text
#     soup = BeautifulSoup(html, "html.parser")

#     # ищем ссылку на последнюю страницу (»»)
#     pagination_links = soup.select("ul.pagination a[aria-label='Next']")

#     if pagination_links:
#         last_page_link = pagination_links[-1]
#         last_page = int(re.search(r"page=(\d+)", last_page_link["href"]).group(1))
#     else:
#         last_page = 1  # Если нет пагинации, значит всего одна страница

#     # случайные страницы без повторов
#     pages = random.sample(range(1, last_page+1), k=min(5, last_page))

#     for random_page in pages:      # количество обработанных страниц 
#         url_page = f'https://www.biblio-globus.ru/catalog/category?id={subcategorie}&page={random_page}'
#         book_list = extract_product_ids(url_page)

#         if not book_list:
#             continue

#         # берём до 5 случайных книг
#         for num in random.sample(range(len(book_list)), k=min(5, len(book_list))):
#             url_book = f'https://www.biblio-globus.ru/product/{book_list[num]}'
#             pars_book_data(url_book, csv_filename="../data/books.csv")
#             time.sleep(random.uniform(0.5, 2))  # пауза чтобы не банили

In [181]:
# import re, time, random, requests
# from bs4 import BeautifulSoup

# for i in range(1000):  # количество заходов по категориям
#     try:
#         n = random.randint(0, len(links) - 1)
#         subcategorie = links.iloc[n][2]

#         url_subcat = f'https://www.biblio-globus.ru/catalog/category?id={subcategorie}'
#         try:
#             html = requests.get(url_subcat, timeout=10).text
#         except Exception as e:
#             print(f"[ERROR] Не удалось загрузить категорию {subcategorie}: {e}")
#             continue

#         soup = BeautifulSoup(html, "html.parser")

#         # ищем ссылку на последнюю страницу (»»)
#         pagination_links = soup.select("ul.pagination a[aria-label='Next']")
#         if pagination_links:
#             last_page_link = pagination_links[-1]
#             last_page = int(re.search(r"page=(\d+)", last_page_link["href"]).group(1))
#         else:
#             last_page = 1  # Если нет пагинации, значит всего одна страница

#         # случайные страницы без повторов
#         pages = random.sample(range(1, last_page+1), k=min(5, last_page))
#         for random_page in pages:  # количество обработанных страниц 
#             url_page = f'https://www.biblio-globus.ru/catalog/category?id={subcategorie}&page={random_page}'

#             try:
#                 book_list = extract_product_ids(url_page)
#             except Exception as e:
#                 print(f"[ERROR] Не удалось получить список книг: {e}")
#                 continue

#             if not book_list:
#                 print(f"[WARNING] Нет книг на странице {url_page}")
#                 continue

#             # берём до 5 случайных книг
#             for num in random.sample(range(len(book_list)), k=min(5, len(book_list))):
#                 try:
#                     url_book = f'https://www.biblio-globus.ru/product/{book_list[num]}'
#                     pars_book_data(url_book, csv_filename="../data/books.csv") # type: ignore
#                 except Exception as e:
#                     print(f"[ERROR] Ошибка парсинга книги {url_book}: {e}")
#                     continue

#                 time.sleep(random.uniform(0.5, 2))  # пауза чтобы не банили

#     except Exception as e:
#         print(f"[ERROR] Ошибка при обработке категории: {e}")
#         continue

# Паралельный парсер

In [179]:
# import os, re, time, random, requests, csv
# from bs4 import BeautifulSoup
# from concurrent.futures import ThreadPoolExecutor, as_completed

# csv_filename = "../data/books.csv"
# fieldnames = ["title", "author", "url", "image", "description"]

# def pars_book_data_return(url):
#     """Парсит страницу и возвращает dict (или None при ошибке)."""
#     try:
#         response = requests.get(url, timeout=10)
#         response.raise_for_status()
#     except Exception as e:
#         print(f"[ERROR] Не удалось загрузить страницу {url}: {e}")
#         return None

#     soup = BeautifulSoup(response.text, "lxml")
#     title = (soup.find("meta", property="og:title") or {}).get("content", "")
#     author = (soup.find("meta", property="og:book:author") or {}).get("content", "")
#     url_value = (soup.find("meta", property="og:url") or {}).get("content", "")
#     image = (soup.find("meta", property="og:image") or {}).get("content", "")

#     description = ""
#     module_div = soup.find('div', id='module')
#     if module_div:
#         collapse_div = module_div.find('div', class_='collapse')
#         if collapse_div:
#             for table in collapse_div.find_all('table'):
#                 table.decompose()
#             for h3 in collapse_div.find_all('h3'):
#                 if 'Характеристики' in h3.get_text():
#                     h3.decompose()
#             description = collapse_div.get_text(strip=True, separator=' ')
#             description = ' '.join([line.strip() for line in description.split('\n') if line.strip()])

#     return {
#         "title": title,
#         "author": author,
#         "url": url_value,
#         "image": image,
#         "description": description
#     }

In [178]:
# for i in range(1000):  # количество заходов по категориям
#     try:
#         n = random.randint(0, len(links) - 1)
#         subcategorie = links.iloc[n][2]

#         url_subcat = f'https://www.biblio-globus.ru/catalog/category?id={subcategorie}'
#         try:
#             html = requests.get(url_subcat, timeout=10).text
#         except Exception as e:
#             print(f"[ERROR] Не удалось загрузить категорию {subcategorie}: {e}")
#             continue

#         soup = BeautifulSoup(html, "html.parser")

#         # ищем ссылку на последнюю страницу (»»)
#         pagination_links = soup.select("ul.pagination a[aria-label='Next']")
#         if pagination_links:
#             last_page_link = pagination_links[-1]
#             last_page = int(re.search(r"page=(\d+)", last_page_link["href"]).group(1))
#         else:
#             last_page = 1

#         # случайные страницы без повторов
#         pages = random.sample(range(1, last_page+1), k=min(5, last_page))
        
#         for random_page in pages:  # количество обработанных страниц 
#             # ДОБАВЬТЕ ЭТУ СТРОКУ - получаем книги для КАЖДОЙ страницы
#             url_page = f'https://www.biblio-globus.ru/catalog/category?id={subcategorie}&page={random_page}'
#             book_list = extract_product_ids(url_page)  # ← ЭТОГО НЕ БЫЛО!
            
#             if not book_list:
#                 continue

            
#             # внутри твоего for random_page in pages:
#             urls_books = [
#                 f'https://www.biblio-globus.ru/product/{book_list[num]}'
#                 for num in random.sample(range(len(book_list)), k=min(5, len(book_list)))
#             ]

#             # парсим параллельно
#             results = []
#             with ThreadPoolExecutor(max_workers=10) as executor:
#                 futures = {executor.submit(pars_book_data_return, u): u for u in urls_books}
#                 for fut in as_completed(futures):
#                     url = futures[fut]
#                     try:
#                         res = fut.result()
#                         if res:
#                             results.append(res)
#                     except Exception as e:
#                         print(f"[ERROR] В потоке при парсинге {url}: {e}")

#             # запись одним батчем (основной поток)
#             if results:
#                 file_new = not os.path.exists(csv_filename) or os.path.getsize(csv_filename) == 0
#                 with open(csv_filename, "a", newline="", encoding="utf-8") as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#                     if file_new:
#                         writer.writeheader()
#                     for res in results:
#                         writer.writerow({k: res.get(k, "") for k in fieldnames})

#             # опционально — небольшая пауза между страницами/батчами
#             time.sleep(random.uniform(0.5, 1.5))

#     except Exception as e:
#         print(f"[ERROR] Ошибка при обработке категории: {e}")
#         continue

In [185]:
data = pd.read_csv('../data/Nadia_books.csv')
data.head()

,title,author,url_value,image,description
0,ИСТОРИК №09 (129). Сентябрь 2025,NaN,https://www.biblio-globus.ru/product/11060070,https://static1.bgshop.ru/imagehandler.ashx?fi...,Читайте в номере: ПУТЬ К ЦАРСТВУ: как Москва и...
1,Тайны XIX века. Тайны морей и островов,Булычев К.,https://www.biblio-globus.ru/product/11021457,https://static1.bgshop.ru/imagehandler.ashx?fi...,"Прошлое надежно хранит свои тайны и, как всем ..."
2,Новый японско-русский и русско-японский слова...,NaN,https://www.biblio-globus.ru/product/10296217,https://static1.bgshop.ru/imagehandler.ashx?fi...,Словарь удобен для быстрого и точного перевода...
3,Японский для новичков,Сыщикова А.Н.,https://www.biblio-globus.ru/product/10977175,https://static1.bgshop.ru/imagehandler.ashx?fi...,Александра Сыщикова — дипломированный специали...
4,Японский язык. Учебный словарь иероглифов. 200...,Попов М.,https://www.biblio-globus.ru/product/10277868,https://static1.bgshop.ru/imagehandler.ashx?fi...,Словарь содержит свыше 2 000 иероглифов. Иерог...


In [186]:
data.isna().sum()

title            21
author         2613
url_value        21
image            21
description     119
dtype: int64

In [187]:
data.shape

(27432, 5)

In [189]:
dupes = data[data.duplicated(subset=["author", "title"], keep=False)]
dupes.shape

(17102, 5)

In [190]:
unique_books = dupes.drop_duplicates(subset=["author", "title"])
print(len(unique_books))  # столько уникальных книг-дубликатов

5690


In [191]:
unique_books_all = data.drop_duplicates(subset=["author", "title"])
print(len(unique_books_all))  # столько уникальных книг-дубликатов

16020


In [192]:
unique_books_all.isna().sum()

title             1
author         1528
url_value         1
image             1
description      94
dtype: int64

#### Обработка пропусков

In [193]:
nans = data[(data["author"] == "") | (data["author"].isna())]
keywords = ["календарь", "Календарь", "Набор"]
pattern = "|".join(keywords)                                   # "календарь|Календарь"
unique_books_all = unique_books_all.drop(
    unique_books_all[unique_books_all["title"].str.contains(pattern, case=False, na=False)].index
)
unique_books_all = unique_books_all.dropna(subset=["description", 'title', 'url_value', 'image'])
unique_books_all["author"] = unique_books_all["author"].fillna("Без автора")

In [194]:
unique_books_all.isna().sum()

title          0
author         0
url_value      0
image          0
description    0
dtype: int64

In [195]:
unique_books_all.shape

(15878, 5)